In [1]:
import pandas as pd
import numpy as np
import datetime
from haversine import haversine
import os
import json
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

print(datetime.datetime.now())

2020-03-02 14:50:24.085836


In [2]:
df_competition_P=pd.read_excel("./Biglots_Competitors_PS_Count_20200214 CC.xlsx",dtype=str,sheet_name="P_counts")
df_competition_S=pd.read_excel("./Biglots_Competitors_PS_Count_20200214 CC.xlsx",dtype=str,sheet_name="S_counts")

df_competition_P=df_competition_P[pd.notnull(df_competition_P['location_id'])]
df_competition_P=df_competition_P[df_competition_P['location_id']!="nan"]
df_competition_S=df_competition_S[pd.notnull(df_competition_S['location_id'])]
df_competition_S=df_competition_S[df_competition_S['location_id']!="nan"]

df_competition_P=df_competition_P.rename(columns={"Competitor Count ":"P_competitors"})
df_competition_S=df_competition_S.rename(columns={"Competitor Count ":"S_competitors"})
df_competition_P['P_competitors']=df_competition_P['P_competitors'].astype(int)
df_competition_S['S_competitors']=df_competition_S['S_competitors'].astype(int)

df_competition=pd.merge(df_competition_P,df_competition_S,on="location_id",how="left")
df_competition['P_competitors']=df_competition['P_competitors'].fillna(0)
df_competition['S_competitors']=df_competition['S_competitors'].fillna(0)

df_competition['competitor_count']=df_competition['P_competitors']+df_competition['S_competitors']
print(df_competition.shape,df_competition['location_id'].nunique())
df_competition['competitor_count'].median()

(1403, 4) 1403


17.0

In [3]:
df_competition.head(3)

,location_id,P_competitors,S_competitors,competitor_count
0,1,3,17,20
1,3,4,15,19
2,29,6,0,6


In [4]:
df_competition=df_competition[['location_id','competitor_count']]

# >=17 high, <=16 low
df_competition['competition_label']=np.where(df_competition['competitor_count']>=17,"H","L")
df_competition=df_competition[['location_id','competition_label']]
print(df_competition.shape,df_competition['location_id'].nunique())

(1403, 2) 1403


In [5]:
df_march_store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200301-134228-900.txt",dtype=str,sep="|")
df_march_store_list=df_march_store_list[['location_id','latitude_meas','longitude_meas']]
df_march_store_list['latitude_meas']=df_march_store_list['latitude_meas'].astype(float)
df_march_store_list['longitude_meas']=df_march_store_list['longitude_meas'].astype(float)
df_march_store_list=df_march_store_list[df_march_store_list['location_id']!="6990"]
df_march_store_list=df_march_store_list[df_march_store_list['location_id']!="145"]

df_march_store_list.head(2)

,location_id,latitude_meas,longitude_meas
0,3,33.421570,-82.018921
1,30,41.363132,-83.650909


In [7]:
df_store_zips=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
print(df_store_zips.sheet_names)
df_store_zips=df_store_zips.parse("view_by_store",dtype=str)
df_store_zips.head(2)

['unique_zips_full_footprint', 'view_by_TA', 'view_by_store', 'view_for_Tableau']


,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [8]:
df_by_store_zips=pd.DataFrame()
for ind,row in df_store_zips.iterrows():
    store_num=row['location_id']
    P_zips=pd.DataFrame({"zip_cd":eval(row['trans_P_zips_70_within_TA'])})
    P_zips['zip_label']="P"
    S_zips=pd.DataFrame({"zip_cd":eval(row['trans_S_zips_70_within_TA'])})
    S_zips['zip_label']="S"
    zips_in_10=pd.DataFrame({"zip_cd":eval(row['zips_in_10'])})
    zips_in_10['zip_label']="zips_in_10"
    
    df=pd.concat([P_zips,S_zips,zips_in_10])
    df['location_id']=store_num
    df_by_store_zips=df_by_store_zips.append(df)
df_by_store_zips=df_by_store_zips[['location_id','zip_cd','zip_label']]

df_output_in_TA=pd.merge(df_march_store_list,df_by_store_zips,on="location_id",how="left")

df_output_new=df_output_in_TA[pd.isnull(df_output_in_TA['zip_cd'])]
df_output_in_TA=df_output_in_TA[pd.notnull(df_output_in_TA['zip_cd'])]


In [9]:
print(df_output_new.shape,df_output_new['location_id'].nunique())
print(df_output_in_TA.shape,df_output_in_TA['location_id'].nunique())

(31, 5) 31
(22575, 5) 1372


In [10]:
df_output_new=df_output_new.reset_index()
del df_output_new['index']
del df_output_new['zip_cd']
del df_output_new['zip_label']

df_new_zips=pd.DataFrame()
for ind,row in df_output_new.iterrows():
    store_center=(row['latitude_meas'],row['longitude_meas'])
    list_store_10_zips=[]
    for zip_cd,v in zip_centers.items():
        dist=haversine(store_center,v,unit="mi")
        if dist<=10:
            list_store_10_zips.append(zip_cd)
    df=pd.DataFrame({"zip_cd":list_store_10_zips})
    df['location_id']=row['location_id']
    df_new_zips=df_new_zips.append(df)
df_new_zips['zip_label']="zips_in_10"
df_new_zips=df_new_zips[['location_id','zip_cd','zip_label']]
df_output_new=pd.merge(df_output_new,df_new_zips,on="location_id",how="left")
df_output_new.head(3)

,location_id,latitude_meas,longitude_meas,zip_cd,zip_label
0,4679,42.723314,-84.424187,48919,zips_in_10
1,4679,42.723314,-84.424187,48912,zips_in_10
2,4679,42.723314,-84.424187,48930,zips_in_10


In [11]:
df_output_new['location_id'].nunique()

31

In [12]:
df_output_in_TA.head(2)

,location_id,latitude_meas,longitude_meas,zip_cd,zip_label
0,3,33.42157,-82.018921,30906,P
1,3,33.42157,-82.018921,30904,S


In [13]:
df_output_zip_store=df_output_in_TA.append(df_output_new)
print(df_output_zip_store['location_id'].nunique())
df_output_zip_store=pd.merge(df_competition,df_output_zip_store,on="location_id",how="outer")
print(df_output_zip_store['location_id'].nunique())

1403
1405


In [14]:
list_stores_filled_na_L_competition=df_output_zip_store[pd.isnull(df_output_zip_store['competition_label'])]['location_id'].unique().tolist()
df_stores_filled_na_L_competition=pd.DataFrame({"stores_without_competition_label":list_stores_filled_na_L_competition})
print(df_stores_filled_na_L_competition.shape)
df_output_zip_store['competition_label']=df_output_zip_store['competition_label'].fillna("L")

(2, 1)


In [15]:
df_SOTF=pd.read_excel("./BL Store List Report from Dom 2020 0131.xlsx",sheet_name="Store List",dtype=str)
df_SOTF=df_SOTF[['Store',"SOTF"]]
df_SOTF=df_SOTF[df_SOTF['SOTF']=="Yes"]
df_SOTF=df_SOTF.rename(columns={"Store":"location_id"})
df_SOTF['SOTF']="SOTF"

In [16]:
df_output_zip_store=pd.merge(df_output_zip_store,df_SOTF,on="location_id",how="left")
df_output_zip_store['SOTF']=df_output_zip_store['SOTF'].fillna("Legacy")
print(df_output_zip_store.shape)

df_output_zip_store.head(3)
df_output_zip_store['current_open_label']=np.where(pd.isnull(df_output_zip_store['latitude_meas']),"not_open","open")

(23352, 7)


In [17]:
writer=pd.ExcelWriter("./BL_zips_by_competition_SOTF_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

for seg, df_group in df_output_zip_store.groupby(['competition_label','SOTF']):
    df_unique_zip=df_group[df_group['current_open_label']=="open"]
    df_unique_zip=df_group[['competition_label','SOTF','zip_cd']].drop_duplicates()
    df_unique_zip.to_excel(writer,"_".join(seg),index=False)
df_output_zip_store.to_excel(writer,"detials",index=False)
df_stores_filled_na_L_competition.to_excel(writer,"stores_fill_L_competition",index=False)
writer.save()  